In [1]:
#This function is used to load the model information from the file, which currently has the following options:
#
#- Reading functions (lin,log,etc.)
#- Read matrix A 
#- Reads matrix B
#- Variables priority vector reading
#- Tradeoff coefficient

# The modelname field is the name of the file model that is used to read the initial file and name the different output files
# The verbose field is a yes/no field that allows you to view the outputs of the

import pandas as pd
import numpy as np

def loadmodelfromfile(modelname, verbose):
    f = open(modelname+'.txt', 'r')
    f.readline()
    f.readline()
    fx = []
    A = []
    B = []
    p = []
    l = '_'
    e = None
    delta = []
    while 1:
        l = f.readline().rstrip()
        if l != '': 
            fx.append(l) 
        else: 
            break
    while 1:
        l = f.readline().rstrip()
        if l != '':
            if l.__contains__(','):
                l = l.split(',')
                l = [float(x) for x in l]
                A.append(l)
            else:    
                A.append(float(l))
        else:
            break
    while 1:
        l = f.readline().rstrip()
        if l != '':
            if l.__contains__(','):
                l = l.split(',')
                l = [float(x) for x in l]
                B.append(l)
            else:    
                B.append(float(l))
        else:
            break
    f.readline()
    f.readline()

    try:
        for x in f.readline().rstrip().split(','):
            p.append(int(x))
    except:
        pass
    f.readline()
    try:
        e = float(f.readline().rstrip())
    except:
        pass
    f.readline()
    f.readline()
    f.readline()
    try:
        for x in f.readline().rstrip().split(','):
            delta.append(float(x))
    except:
        pass

    if verbose:
        print("Model: %s" % modelname)
        print("Function list: ")
        print(fx) 
        print("A matrice: ")
        print(A)
        print("B matrice: ")
        print(B)
        print("Priority factor array (most important to least important): ")
        print(p)
        print("Trade-off coeficiente ")
        print(e)
        print("Tolerances: ")
        print(delta)

    return fx, A, B, p, e, delta

#Test1
#modelname = 'Modelo2'
#m_fx = []
#m_A = []
#m_B = []
#m_p = []
#m_fx,m_A,m_B,m_p,m_e,m_delta = loadmodelfromfile(modelname, False)

In [2]:
#In this cell are the functions to convert the values from the original scale to the value scale and vice-versa at the end to display the results
from itertools import count
import numpy as np
import pandas as pd
import math

def lin(lin_y,lin_X,lin_Y):
    lin_m = (lin_Y[1]-lin_Y[0])/(lin_X[1]-lin_X[0])
    lin_b = lin_Y[0] - lin_m*lin_X[0]
    lin_x = (lin_y - lin_b)/lin_m
    return lin_x
             
#Functions to convert to value scale
def converttovaluescale(el,uf_i,uf_m, A, B):
    match uf_m:
        case 'lin':
            reverse_A = A[uf_i].copy()
            reverse_A.sort(reverse=True)
            index = A[uf_i].index(min(reverse_A,key=lambda x : x - el > 0 ))
            A_uf_i = np.array(A[uf_i])
            B_uf_i = np.array(B[uf_i])
            if index == len(A[uf_i])-1:
                return B_uf_i[index]
            else:
                return (lin(el,B_uf_i[index:index+2],A_uf_i[index:index+2])) 
        case 'exp':
            return A[uf_i]*math.exp(el)+B[uf_i]
        case 'log':
            return A[uf_i]*math.log(el)+B[uf_i]
        case _:
            return -999

#Convert to value scale
def convertoriginaltovaluescale(modelname, fx, A, B):
    df1 = pd.read_csv(modelname+'_originals.csv')
    n = len(fx)
    for i in range(1,n+1):
        df1.iloc[:, i] = df1.iloc[:, i].apply(lambda x: converttovaluescale(x,i-1,fx[i-1],A,B))
    df1_headers = ['DMUs']
    for i in range(1,n+1):
        df1_headers.append('V'+str(i))
    df1.columns = df1_headers
    df1.to_csv(modelname+"_R_originals_valuescale.csv",float_format='%.12f', index=False)
    return df1

#Functions to convert back to original scale    
def converttooriginalscale(el,uf_i,uf_m,A,B):    
    if pd.isna(el): return el
    match uf_m:
        case 'lin':
            if B[uf_i][0]>B[uf_i][1]:
                A[uf_i].sort(reverse=False)
                B[uf_i].sort(reverse=False)
            reverse_B = B[uf_i].copy()
            reverse_B.sort(reverse=True)            
            index = B[uf_i].index(min(reverse_B,key=lambda x : x - el > 0 ))
            A_uf_i = np.array(A[uf_i])
            B_uf_i = np.array(B[uf_i])
            if index == len(B[uf_i])-1:
                return A_uf_i[index]
            else:
                return (lin(el,A_uf_i[index:index+2],B_uf_i[index:index+2]))        
        case 'exp':
            return math.log((el-B[uf_i])/A[uf_i])
        case 'log':
            return math.exp((el-B[uf_i])/A[uf_i])
        case _:
            return -999

#Convert back to original scale
def convertvaluescaletooriginal(modelname,df_phase2,fx,A,B):
    df = pd.read_csv(modelname+'_R_phase2.csv')
    n = len(fx)
    for i in range(1,n+1):
        df.iloc[:, i+1] = df.iloc[:, i+1].apply(lambda x: converttooriginalscale(x,i-1,fx[i-1],A,B))
    for i in range(n+1,2*n+1):
        df.iloc[:, i+1] = df.iloc[:, i+1].apply(lambda x: converttooriginalscale(x,i-1-(n+1),fx[i-1-(n+1)],A,B))
    df.to_csv(modelname+"_R_final.csv", float_format='%.12f',index=False)


#Test2
#df_vscale = convertoriginaltovaluescale(modelname, m_fx , m_A, m_B)

In [3]:
#Function for robustness analysis
def createbounds(modelname,delta):
    df1 = pd.read_csv(modelname+'_originals.csv')
    df3 = pd.DataFrame()
    df3['DMUs'] = df1['DMUs']
    for i in range(1,len(df1.columns)):
        df3['LB'+str(i)] = df1.iloc[:,i].apply(lambda x: x*(1-delta) if x > 0 else x*(1+delta))
        df3['UB'+str(i)] = df1.iloc[:,i].apply(lambda x: x*(1+delta) if x > 0 else x*(1-delta))
    df3.to_csv(modelname+"_R_"+str(delta)+"_bounds.csv",float_format='%.12f', index=False)
    return df3    

def convertboundsoriginaltovaluescale(modelname, df_robustness_original, fx_delta, A_delta, B_delta, delta):
    df1 = pd.read_csv(modelname+'_R_'+str(delta)+'_bounds.csv')
    n = len(fx_delta)
    for i in range(1,n+1):
        df1.iloc[:, i] = df1.iloc[:, i].apply(lambda x: converttovaluescale(x,i-1,fx_delta[i-1],A_delta,B_delta))
    for i in range(len(df1)):
        for j in range(1,len(df1.columns),2):
            if (df1.iloc[i,j]>df1.iloc[i,j+1]):
                _aux = df1.iloc[i,j]
                df1.iloc[i,j] = df1.iloc[i,j+1]
                df1.iloc[i,j+1] = _aux
    df1.to_csv(modelname+'_R_'+str(delta)+"_bounds_valuescale.csv",float_format='%.12f', index=False)
    return df1  
 

In [4]:
#Phase one for robustness analysis
#Read several dealtas (0.05,0.10,etc.) and present data in columns (DMU, d_opt(para 5% - esq), d_pess(5% - dir))
import numpy as np
import pandas as pd
from scipy.optimize import linprog

def phase1_robustness_analysis(modelname,load,df_robustness_vscale,p,e,delta):
    #Load values
    if load:
        df_robustness_vscale = pd.read_csv(modelname+'_R_bounds_valuescale.csv') 
    df_vscale2 = df_robustness_vscale.iloc[:,1:]
    nw = int(len(df_vscale2.columns)/2)
    ndmu = len(df_vscale2)
    _A = df_vscale2.to_numpy()
    _Aesq = np.zeros((ndmu,int(nw)))
    _Adir = np.zeros((ndmu,int(nw)))

    for i in range(ndmu):
        for j in range(0,nw):
            _Aesq[i,j] = _A[i,2*j]
            _Adir[i,j] = _A[i,2*j+1] 

    res_fase1_esq = []
    for i in range(ndmu):
        c0 = np.zeros(nw)
        c = np.append(c0,1)

        A2 = np.copy(_Aesq)
        A_27 = np.copy(_Adir[i])
        A2[i] = np.zeros(nw)
        A3 = A2 - A_27.T

        A_ub = np.c_[A3,-1*np.ones((ndmu,1))]
        b_ub = np.zeros(ndmu)

        A_eq = []
        b_eq = []
        A_eq0 = np.ones(nw)
        A_eq0 = np.append(A_eq0,0)       
        A_eq.append(A_eq0.tolist())
        b_eq.append(1)

        if p:
            for i_p in range(len(p)-1):
                A_ub1 = np.zeros(nw+1)
                A_ub1[p[i_p]-1] = -1
                A_ub1[p[i_p+1]-1] = 1
                A_ub2 = []
                A_ub2.append(A_ub1.tolist())
                A_ub = np.r_[A_ub,np.array(A_ub2)]
                b_ub = np.append(b_ub,0)
            if e:
                A_ub3 = np.zeros(nw+1) 
                A_ub3[p[len(p)-1]-1] = -1*e   
                A_ub3[p[0]-1] = 1
                A_ub4 = []
                A_ub4.append(A_ub3.tolist())
                A_ub = np.r_[A_ub,np.array(A_ub4)]
                b_ub = np.append(b_ub,0)
        
        wn_bounds = []
        for j in range(nw):
            wn_bounds.append((0,None))
        wn_bounds.append((None,None))
        res = linprog(c, A_ub=A_ub, b_ub=b_ub,A_eq=A_eq, b_eq=b_eq,bounds=wn_bounds)
        res2 = np.zeros(nw+2)
        res2[0] = i+1
        res2[1:] = res.x
        res_fase1_esq.append(res2)
    
    column_names = []
    column_names.append('DMUs')
    df2 = pd.DataFrame(data = df_robustness_vscale['DMUs'], columns=column_names)
    df2['d_'+str(delta)+'_opt'] = np.array(res_fase1_esq)[:,-1].tolist()

    res_fase1_dir = []
    for i in range(ndmu):
        c0 = np.zeros(nw)
        c = np.append(c0,1)

        A2 = np.copy(_Adir)
        A_27 = np.copy(_Aesq[i])
        A2[i] = np.zeros(nw)
        A3 = A2 - A_27.T

        A_ub = np.c_[A3,-1*np.ones((ndmu,1))]
        b_ub = np.zeros(ndmu)

        A_eq = []
        b_eq = []
        A_eq0 = np.ones(nw)
        A_eq0 = np.append(A_eq0,0)       
        A_eq.append(A_eq0.tolist())
        b_eq.append(1)

        if p:
            for i_p in range(len(p)-1):
                A_ub1 = np.zeros(nw+1)
                A_ub1[p[i_p]-1] = -1
                A_ub1[p[i_p+1]-1] = 1
                A_ub2 = []
                A_ub2.append(A_ub1.tolist())
                A_ub = np.r_[A_ub,np.array(A_ub2)]
                b_ub = np.append(b_ub,0)
            if e:
                A_ub3 = np.zeros(nw+1) 
                A_ub3[p[len(p)-1]-1] = -1*e   
                A_ub3[p[0]-1] = 1
                A_ub4 = []
                A_ub4.append(A_ub3.tolist())
                A_ub = np.r_[A_ub,np.array(A_ub4)]
                b_ub = np.append(b_ub,0)
        
        wn_bounds = []
        for j in range(nw):
            wn_bounds.append((0,None))
        wn_bounds.append((None,None))

        res = linprog(c, A_ub=A_ub, b_ub=b_ub,A_eq=A_eq, b_eq=b_eq,bounds=wn_bounds)
        res2 = np.zeros(nw+2)
        res2[0] = i+1
        res2[1:] = res.x
        res_fase1_dir.append(res2)
    
    df2['d_'+str(delta)+'_pes'] = np.array(res_fase1_dir)[:,-1]
    df2.to_csv(modelname+"_R_"+str(delta)+"_bounds_phase1.csv", float_format='%.12f', index = False)

    return df2

#Run model
#modelname = 'Modelo2'
#df_phase1 = phase1_robustness_analysis(modelname,False,df_robustness_vscale,m_p,m_e,m_delta[0])
#print(df_phase1.head(10))

In [6]:
#This main program runs robustness
import pandas as pd
modelname = 'Modelo2'
#Load mdodel from file  
m_fx = []
m_A = []
m_B = []
m_p = []
#loadmodelfromfile(modelname, verbose)
m_fx,m_A,m_B,m_p,m_e,m_delta = loadmodelfromfile(modelname, False)

for m_delta_i in m_delta:
    df_robustness_original = createbounds(modelname, m_delta_i)
    m_A_delta = []
    m_B_delta = []
    m_fx_delta = []
    for i in range(len(m_fx)):
        m_A_delta.append(m_A[i])
        m_A_delta.append(m_A[i])
        m_B_delta.append(m_B[i])
        m_B_delta.append(m_B[i])
        m_fx_delta.append(m_fx[i])
        m_fx_delta.append(m_fx[i])
      
        df_robustness_vscale = convertboundsoriginaltovaluescale(modelname, df_robustness_original, m_fx_delta, m_A_delta, m_B_delta, m_delta_i)
        phase1_robustness_analysis(modelname,False,df_robustness_vscale,m_p,m_e,m_delta_i)

In [11]:
#Imprimir a pesquisa binaria para descobrir o delta maximo
#Resposta com o delta e os valores das eficienciais
#Só em relação as DMUS com valroes de eficiencia que variam entre (negativo e positivo)
#delta a variar de 0 a 0.25
#apanhar o pessimista
#Para cada DMU calcular percorrer os desltas calculando as bounds e ir incrementado até que o pessimista seja negativo

def determinardeltasparaDMUSinefecientes(modelname,fx,A,B,p,e):
    delta = 0.01
    dmus_que_ja_apareceram = []
    for delta_i in range(200):
        delta += 0.001
        df_robustness_original = createbounds(modelname,delta)
        m_A_delta = []
        m_B_delta = []
        m_fx_delta = []
        for i in range(len(fx)):
            m_A_delta.append(A[i])
            m_A_delta.append(A[i])
            m_B_delta.append(B[i])
            m_B_delta.append(B[i])
            m_fx_delta.append(fx[i])
            m_fx_delta.append(fx[i])
        df_robustness_vscale = convertboundsoriginaltovaluescale(modelname, df_robustness_original, m_fx_delta, m_A_delta, m_B_delta, delta)
        df_robustness_vscale_phase1 = phase1_robustness_analysis(modelname,False,df_robustness_vscale,p,e,delta)
        #Apos a primeira etapa para 0.05 vai correr um algoritmo simular para todas as DMUS com pess negativo e optimo positivo
        for i in range(len(df_robustness_vscale_phase1)):
            if df_robustness_vscale_phase1.iloc[i,1] < 0 and df_robustness_vscale_phase1.iloc[i,2] > 0 and df_robustness_vscale_phase1.iloc[i,0] not in dmus_que_ja_apareceram:
                #testar varios deltas ate encontrar o delta maximo    
                dmus_que_ja_apareceram.append(df_robustness_vscale_phase1.iloc[i,0])
                print("Delta: ", delta, " DMU:", df_robustness_vscale_phase1.iloc[i,0])



#This main program runs robustness
import pandas as pd
modelname = 'Modelo2'
#Load mdodel from file  
m_fx = []
m_A = []
m_B = []
m_p = []
#loadmodelfromfile(modelname, verbose)
m_fx,m_A,m_B,m_p,m_e,m_delta = loadmodelfromfile(modelname, False)
determinardeltasparaDMUSinefecientes(modelname,m_fx,m_A,m_B,m_p,m_e)

Delta:  0.011  DMU: HU
Delta:  0.011  DMU: IT
Delta:  0.012  DMU: DK
Delta:  0.013000000000000001  DMU: SK
Delta:  0.018000000000000006  DMU: PL
Delta:  0.019000000000000006  DMU: IE
Delta:  0.02400000000000001  DMU: CZ
Delta:  0.03800000000000002  DMU: FR
Delta:  0.044000000000000025  DMU: UK
Delta:  0.054000000000000034  DMU: DE
Delta:  0.05700000000000004  DMU: LV
Delta:  0.05700000000000004  DMU: RO
Delta:  0.10000000000000007  DMU: AT
Delta:  0.10000000000000007  DMU: BE
Delta:  0.10000000000000007  DMU: MT
Delta:  0.10000000000000007  DMU: SE
Delta:  0.10100000000000008  DMU: BG
Delta:  0.10400000000000008  DMU: CY
Delta:  0.10900000000000008  DMU: GR
Delta:  0.1420000000000001  DMU: LU
